In [29]:
"""Create parameters.py from a .inp and load it"""
# This cell is facultative, you can use an existing parameters.py

# Directory for .inp file:
path = 'F:\\OneDrive - Budapesti Műszaki és Gazdaságtudományi Egyetem\\TDK\\chemkin_AR_HE.inp'

# import libraries:
import importlib   # for reloading your own files
# my own files:
import inp_data_extractor as inp
importlib.reload(inp)

# create parameters.py
inp.extract(path)

# load parameters.py
import parameters as par
importlib.reload(par)
print(par.model)

Note, lambda value for specie 'H' is not in data.py: 0.0 is used
Note, lambda value for specie 'O' is not in data.py: 0.0 is used
Note, lambda value for specie 'OH' is not in data.py: 0.0 is used
Note, lambda value for specie 'HO2' is not in data.py: 0.0 is used
Note, lambda value for specie 'OHEX' is not in data.py: 0.0 is used
Warning, third body 'CO' is not in species in line 65 ('H2/2.5/ O2/1/ H2O/12/ CO/1.9/ CO2/3.8/ HE/0.83/ AR/1/') in reaction '2H+M=H2+M'
Warning, third body 'CO2' is not in species in line 65 ('H2/2.5/ O2/1/ H2O/12/ CO/1.9/ CO2/3.8/ HE/0.83/ AR/1/') in reaction '2H+M=H2+M'
Warning, third body 'CO' is not in species in line 67 ('H2/2.5/ H2O/12/ AR/0.83/ CO/1.9/ CO2/3.8/ HE/0.83/') in reaction '2O+M=O2+M'
Warning, third body 'CO2' is not in species in line 67 ('H2/2.5/ H2O/12/ AR/0.83/ CO/1.9/ CO2/3.8/ HE/0.83/') in reaction '2O+M=O2+M'
Warning, third body 'CO' is not in species in line 69 ('H2/2.5/ O2/1/ H2O/12/ AR/0.75/ CO/1.5/ CO2/2/ HE/0.75/') in reaction 'O+H

In [30]:
"""Libraries"""

# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

import numpy as np   # matrices, math
import time   # runtime measurement
from multiprocessing import Pool, cpu_count   # multithreading
import importlib   # reload changes you made

# my own file:
import full_bubble_model as de    # full bubble model
importlib.reload(de)   # reload changes you made

model: chemkin_AR_HE


<module 'full_bubble_model' from 'F:\\OneDrive - Budapesti Műszaki és Gazdaságtudományi Egyetem\\TDK\\Letisztázott programok\\full_bubble_model.py'>

In [49]:
"""Control parameter ranges and division"""
# a list for each control parameter, containing all the possible values

ranges = de.dotdict(dict(
    # Equilibrium radius [um --> m]
    R_E = [1e-6*x for x in [1.0, 2.0, 3.0, 4.0, 5.0, 7.5, 10.0, 15.0, 20.0, 25.0]],
    # R_star / R_E [-]
    ratio = [2.0, 3.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0],
    # Ambient pressure [Pa]
    P_inf = [1e5*x for x in [1.0, 10.0, 25.0, 50.0]],
    # Accommodation coeff. for evaporation [-]
    alfa_M = [0.05],
    # Ambient temperature [°C --> K]
    T_inf = [273.15+x for x in [30.0]],
    # Surface tension modifier [-]
    surfactant = [1.0]
))

# print total combinations:
for key in ranges:
    print(f'{key}: {len(ranges[key])}')
print(f'total combinations: {np.prod([len(ranges[key]) for key in ranges])}')

R_E: 10
ratio: 10
P_inf: 4
alfa_M: 1
T_inf: 1
surfactant: 1
total combinations: 400


In [50]:
"""Get all combinations"""
# Make a list, with one dictionary for eachy parameter combinations

cpars = []
ID = 1
for R_E in ranges.R_E:
    for ratio in ranges.ratio:
        for P_inf in ranges.P_inf:
            for alfa_M in ranges.alfa_M:
                for T_inf in ranges.T_inf:
                    for surfactant in ranges.surfactant:
                        cpar = dict(
                            ID=ID,
                            R_E=R_E, # [m]
                            ratio=ratio, # [-]
                            P_inf=P_inf, # [Pa]
                            alfa_M=alfa_M, # [-]
                            T_inf=T_inf, # [m]
                            surfactant=surfactant, # [-]
                            gases=[de.par.index['AR'], de.par.index['HE']], # indexes of species in initial bubble
                            fractions=[1.0, 0.0] # molar fractions of species in initial bubble
                        )
                        cpar['P_v'] = de.VapourPressure(cpar['T_inf']) # [Pa]
                        cpar['mu_L'] = de.Viscosity(cpar['T_inf']) # [Pa*s]
                        cpars.append(cpar)
                        ID += 1
                        
# Create input dictionary for de.simulate(), a list of dictionaries with cpar and other arguments
kwargs_list = [dict(cpar=cpar, t_int=np.array([0.0, 1.0]), LSODA_timeout=300, Radau_timeout=30) for cpar in cpars]

In [51]:
"""Parameter study, multithread"""
# Runs each combinations in cpars, and saves the results into CSV files
# use Pool(processes=cpu_count()-1) to limit number of threads being used.
# use pool.imap(...) instead of pool.imap_unordered(...) to preserve order in which cpars was made

max_lines = 100    # maximum length of a CSV
best_energy = 1e30

start = time.time()
file = de.Make_dir('test 1')
file.new_file()
with Pool(processes=cpu_count(), maxtasksperchild=100) as pool:
    results = pool.imap_unordered(de.simulate, kwargs_list)

    for data in results:
      # save results:
        if file.lines > max_lines:
            file.close()
            file.new_file()
        data = de.dotdict(data)
        file.write_line(data)
      # print stuff:
        if data.energy>0 and data.energy<best_energy:
            best_energy = data.energy
        print(f'index: {data.ID}/{len(cpars)},   error_code: {data.error_code},   steps: {data.steps},   runtime: {data.elapsed_time: .2f} [s]   |   '+
              f'R_E={1e6*data.R_E: .2f} [um]; ratio={data.ratio: .2f} [-]; P_inf={1e-5*data.P_inf: .2f} [bar]; alfa_M={data.alfa_M: .2f} [-]; '+
              f'T_inf={data.T_inf-273.15: .2f} [°C]; surfactant={100*data.surfactant: .0f} [%]   |   hydrogen production: {data.energy: .1f} [MJ/kg] '+
              f'(best: {best_energy: .1f} [MJ/kg])                                                 ', end='\r')
            
file.close()
end = time.time()
elapsed = end - start
print(f'\n\nDONE')
print(f'total time: {(elapsed / 3600): .0f} hours {((elapsed % 3600) / 60): .0f} mins')
print(f'            {elapsed: .2f} [s]   ({(elapsed / len(cpars)): .2f} [s/run])')

Folder already exists with 0 csv in it
index: 372/400,   error_code: 2,   steps: 14276,   runtime:  310.11 [s]   |   R_E= 25.00 [um]; ratio= 5.00 [-]; P_inf= 50.00 [bar]; alfa_M= 0.05 [-]; T_inf= 30.00 [°C]; surfactant= 100 [%]   |   hydrogen production:  84012.5 [MJ/kg] (best:  10327.4 [MJ/kg])                                                                                              

DONE
total time:  0 hours  9 mins
             531.73 [s]   ( 1.33 [s/run])
